In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# 경고가 많아지면 곤란해서 추가
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# csv 파일 리스트 가져오기
files = os.listdir("../csv_data/use/normal/")
path = "../csv_data/use/normal/"

#files

In [ ]:
# concat의 기초가 되어줄 데이터프레임
# 이자보상배율

df_a = pd.read_csv("../raw_data/전체데이터_2011_코스피코스닥_v1.3.csv", thousands=',', encoding="CP949").reset_index(drop=True)
df_a

In [ ]:
# csv 하나 하나 불러오면서 저 옆에 붙입니다

for file in files:
    df_b = pd.read_csv(path+file, thousands=",", encoding="CP949")
    df_b.drop(columns=["Unnamed: 0"], inplace=True)

    df_a = pd.concat([df_a, df_b.iloc[:,-2:]], axis=1)

In [ ]:
df_temp = pd.read_csv("../csv_data/use/special/01_생산성_OK.csv", thousands=",", encoding="CP949")
df_a = pd.concat([df_a, df_temp.iloc[:,-16:]], axis=1)

In [ ]:
df_temp_b = pd.read_csv("../csv_data/use/special/01_활동성_OK.csv", thousands=",", encoding="CP949")
df_a = pd.concat([df_a, df_temp_b.iloc[:,-16:]], axis=1)

In [ ]:
# 나중에 재고자산_유동자산비율 컬럼은 버려!!!
# 개별재무제표와 연결재무제표 중 비교해서 집어넣는 작업을 위한 전처리

df = pd.DataFrame(columns= ["회사명", "거래소코드", "회계년도", '이자보상배율', 'cashflow_매출액비율', 'cashflow_부채비율',
       '단기차입금대총자입금비율', '당좌비율', '당좌자산구성비율', '매입채무회전율', '매출액순이익률', '매출액총이익률', '매출액영업이익률', '매출액증가율',
       '매출원가대매출액비율', '매출채권대매입채무비율', '매출채권회전율', '부채비율', '순운전자본대총자산', '순운전자본회전율',
       '순이익증가율', '영업이익증가율', '운전자본회전비율', '유동부채', '유동비율', '유동자산구성비율', '유동자산증가율', '유동자산회전율',
       '유형자산증가율', '유형자산회전율', '자기자본구성비율', '자본증가율',
       '타인자본회전율', '재고자산대유동자산비율', '재고자산_유동자산비율', '재고자산증가율', '재고자산회전율',
       '차입금비율', '차입금의존도', '차입금평균이자율', '총자본영업이익률', '총자산순이익률',
       '총자산증가율', '총자산회전율', '타인자본구성비율', '현금비율', '노동장비율', '기계장비율', '자본집약도',
       '노동소득분배율', '설비투자효율', '총자본투자효율', '기계투자효율', '부가가치율',
       '총자본회전율', '재고자산회전율', '매출채권회전율', '유형자산회전율',
       '매입채무회전율', '자기자본회전율', '타인자본회전율', '유동자산회전율'])

df["회사명"] = df_a["회사명"]
df["거래소코드"] = df_a["거래소코드"]
df["회계년도"] = pd.to_datetime(df_a['회계년도'].str.upper(), format="%d-%b", yearfirst=False).dt.strftime("20%d")
df

In [ ]:
# csv 한바퀴 돌립니다

df_ind = pd.DataFrame(columns= ["회사명", "거래소코드", "회계년도"])
df_oth = df_ind

for i in range(3, len(df_a.columns)-1, 2):

    # 개별재무제표 저장
    df_x = pd.DataFrame(df_a.iloc[:, i])
    df_ind = pd.concat([df_ind, df_x], axis=1)

for i in range(4, len(df_a.columns), 2):

    # 연결재무제표 저장
    df_y = pd.DataFrame(df_a.iloc[:, i])
    df_oth = pd.concat([df_oth, df_y], axis=1)

In [ ]:
for i in range(0, len(df)):
    # 컬럼들 순회합니다
    for j in range(0, len(df.columns)):
        # df_ind 에 결측치가 없다면
        if not pd.isna(df_ind.iloc[i][j]):
            # df_oth 에도 결측치가 없다면
            if not pd.isna(df_oth.iloc[i][j]):
                # df_oth 삽입
                df.iloc[i, j] = df_oth.iloc[i][j]

            # df_oth 가 결측치라면
            else:
                #df_ind 값 삽입
                df.iloc[i, j] = df_ind.iloc[i][j]

        # df_ind 에 결측치가 있다면
        else:
            # df_oth 에 결측치가 없다면
            if not pd.isna(df_oth.iloc[i][j]):
                # df_oth 삽입
                df.iloc[i, j] = df_oth.iloc[i][j]

            # 둘 다 결측치면 일단 패스
            else:
                pass

In [ ]:
# df.to_csv("../csv_data/00_합체_확인.csv", encoding="CP949")

In [ ]:
# 현재 데이터 기준으로 2년 연속 이.보.배 1 미만인 기업을 찾을 겁니다

#df['이자보상배율'] = df['이자보상배율'].replace(",",'') # 아오

f_company = [] # 2년 연속 이.보.배 1 미만 기업을 저장하는 리스트
f_year = [] # 2년째 최초 연도
count = 0 # 한계기업 카운트용
company = 'x' # 한계기업 기록용

# 이거도 csv 한바퀴 돌립니다
for i in range(0, len(df)):

    # 회사명이 이미 기록된 상태에서 또 중복으로 적을 필요는 없습니다
    if company == df.iloc[i]['회사명']:
        continue
    if df.iloc[i]['이자보상배율'] == 'X':
            continue
    else:
        # print(df.iloc[i]['이자보상배율'])
        # 이자보상배율이 1 미만인 경우 카운트를 늘립니다
        if float(df.iloc[i]['이자보상배율']) < 1:
                count += 1

        # 한계 기업 클럽에 오셨습니다. 환영합니다.
        if count == 2:
            company = df.iloc[i]['회사명']
            f_company.append(company)
            f_year.append(df.iloc[i]['회계년도'])
            count = 0
    #print("반복문 1회")

In [ ]:
f_company = [] # 부실징후기업 저장하는 리스트
f_company_x = [] # 2년 연속 이자보상배율 1 미만인 기업 삽입
count = 0 # 부실징후기업 카운트용
company = 'x' # 부실징후기업 기록용

# 이거도 csv 한바퀴 돌립니다
for i in range(0, len(df)):

    # 회사명이 이미 기록된 상태에서 또 중복으로 적을 필요는 없습니다
    if company == df.iloc[i]['회사명']:
        # 다만 그 기업이 부실징후기업인데
        if count > 1:
            # 어머? 2년 연속이네요? 기록!!!
            if df.iloc[i]['이자보상배율'] < 1:
                f_company_x.append(company)
                count = 0

                # 반복문에서 볼일 다 끝났으니 통과합니다
                continue

        # 그게 아니라면 반복문을 통과합니다
        continue

    # 이자보상배율이 1 미만인 경우 카운트를 늘립니다
    if df.iloc[i]['이자보상배율'] < 1:
        # 그리고 일단 부실징후기업이니 기록은 해야겠죠?
        company = df.iloc[i]['회사명']
        f_company.append(company)
        count += 1

    # 아니면 카운트를 초기화해줍니다
    else:
        count = 0

In [ ]:
limit_c = df.isin(f_company)
limit_c

In [ ]:
df_temp_list = [] # 임시 저장용 리스트

# 그렇게 만든 데이터프레임을 기존과 비교해서 True인 애들만 뽑아옵니다
for i in range(0, len(limit_c)-1):
    if limit_c.iloc[i]["회사명"] == True:   
        df_temp_list.append(df.loc[i])

# 그리고 저장을 합니다
limit_company = pd.DataFrame(df_temp_list).reset_index(drop=True)
limit_company.to_csv("../csv_data/00_부실징후기업v2_0.csv", encoding="CP949")

In [ ]:
limit_d = df.isin(f_company_x)
df_temp_list = [] # 임시 저장용 리스트

# 그렇게 만든 데이터프레임을 기존과 비교해서 True인 애들만 뽑아옵니다
for i in range(0, len(limit_d)-1):
    if limit_d.iloc[i]["회사명"] == True:
        df_temp_list.append(df.loc[i])

# 그리고 저장을 합니다
limit_company = pd.DataFrame(df_temp_list).reset_index(drop=True)
limit_company.to_csv("../csv_data/00_부실확정기업v2_0.csv", encoding="CP949")